# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os

import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig

from azureml.core import Dataset

In [ ]:
print(azureml.core.VERSION)

## Dataset

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:

ws = Workspace.from_config()
ws.write_config(".azureml")

experiment_name = 'udacity-capstone-project'

experiments = Experiment.list(ws, experiment_name=experiment_name)
if not experiments:
      experiment = Experiment(workspace=ws, name=experiment_name)
else:
      experiment = experiments[0]


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster or reuse it if it is available.
cpu_cluster_name = "cpu-cluster-01vx"
vm_size = "Standard_D3_V2"
min_nodes = 0
max_nodes = 6

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster. use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           min_nodes=min_nodes,
                                                           max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# a detailed status for the current cluster.
print(compute_target.get_status().serialize())

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.  
Logistic regression was used to build the classification model. 
Two hyperparameters of the inverse of regularization (C) and the maximum number of iterations (max-iter) were chosen to run HyperDrive.
C is chosen to control the effective size of parameters.
max-iter was tested to determine how long the training should be done to find an optimal model.
For each run, the two parameters were randomly selected (RandomParameter Sampling).
For the early stopping policy, we chose BanditPolicy that seems to be efficient of choosing the early stopping point.

In [ ]:
import os
import shutil

from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

In [ ]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.2, evaluation_interval=10, delay_evaluation=5) 

# Create the different params that you will be using during training
parameter_space = { "--C": uniform(0.1, 1.0), 
                    "--max_iter": choice([50, 100, 500, 1000]) }
param_sampling = RandomParameterSampling(parameter_space)



In [ ]:
script_folder = "training"
os.makedirs(os.path.join(".", script_folder), exist_ok=True)

# Create a SKLearn estimator for use with train.py
shutil.copy("./train.py", os.path.join(".", script_folder))
source_directory=os.path.join(".", script_folder)

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [ ]:
# Use ScrptRunConfig
src = ScriptRunConfig(source_directory=source_directory,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig
hyperdrive_run_config = HyperDriveConfig(
                        run_config=src,
                        hyperparameter_sampling=param_sampling,
                        policy=early_termination_policy,
                        primary_metric_name="accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=30,
                        max_concurrent_runs=max_nodes-1
                        ) 

In [ ]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()["runDefinition"]["arguments"]

In [ ]:
print("Best Run Id      :", best_run.id)
print("Accuracy         :", best_run_metrics["accuracy"])
print("--C              :", parameter_values[1])
print("--max-iter       :", parameter_values[3])

print(parameter_values)

In [ ]:
best_run_filenames = best_run.get_file_names()
for filename in best_run_filenames:
    print(filename)

In [ ]:
# Save the best model
# The last element of best_run_filenames is a joblib file.
best_run.download_file(best_run_filenames[-1], output_file_path=os.path.join(".", script_folder))